In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np

In [3]:
train_data = pd.read_csv("/content/drive/MyDrive/data/train.csv/train.csv")

In [4]:
train_data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [5]:
train_data.shape

(20800, 5)

In [6]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [7]:
train_data.dropna(inplace = True)

In [8]:
train_data.shape

(18285, 5)

In [9]:
train_data.columns

Index(['id', 'title', 'author', 'text', 'label'], dtype='object')

In [10]:
X_train = train_data.drop(['label'], axis = 1)
X_train

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [11]:
y_train = train_data['label']
y_train

0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 18285, dtype: int64

In [15]:
y_train.value_counts()

label
0    10361
1     7924
Name: count, dtype: int64

In [16]:
import re
import nltk
from nltk.corpus import stopwords

In [17]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [18]:
X_train.reset_index(inplace = True)

In [19]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(X_train)):
    if i%2000 == 0:
        print(f"Stemming done for {i} news titles.")

    review = re.sub('[^a-zA-Z]', ' ', X_train['title'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

Stemming done for 0 news titles.
Stemming done for 2000 news titles.
Stemming done for 4000 news titles.
Stemming done for 6000 news titles.
Stemming done for 8000 news titles.
Stemming done for 10000 news titles.
Stemming done for 12000 news titles.
Stemming done for 14000 news titles.
Stemming done for 16000 news titles.
Stemming done for 18000 news titles.


In [20]:
corpus[:20]

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [21]:
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf

In [22]:
vocab_size = 20000

In [23]:
one_hot_rep = [one_hot(word, vocab_size) for word in corpus]
one_hot_rep[:20]

[[11853, 10790, 12699, 16649, 16459, 12453, 15103, 1540, 4706, 4532],
 [18660, 3537, 10293, 11153, 12227, 19888, 14885],
 [5475, 16444, 11841, 17729],
 [14222, 12235, 11473, 1608, 9543, 8267],
 [1744, 12227, 6464, 12445, 4656, 5152, 12227, 12488, 11281, 321],
 [11121,
  9304,
  6137,
  17487,
  14126,
  545,
  16468,
  8953,
  18079,
  10234,
  7170,
  18342,
  1124,
  4063,
  14885],
 [11880, 12551, 5851, 5507, 13889, 5090, 663, 9619, 5566, 8129, 6950],
 [10360, 11758, 19373, 1414, 10032, 13619, 545, 1292, 5566, 8129, 6950],
 [17105, 723, 1698, 6815, 8705, 12781, 18077, 12405, 545, 15665],
 [14226, 3456, 790, 9042, 14360, 12830, 13131, 18471],
 [4174, 19280, 6186, 15403, 3603, 5089, 2574, 5636, 93, 14573, 13680],
 [1608, 5040, 16459, 12781, 545, 10032],
 [2591, 9184, 19507, 18634, 5452, 16704, 17821, 4450, 8447],
 [284, 8452, 10340, 6123, 5284, 18775, 15032, 5566, 8129, 6950],
 [13107, 3877, 158, 1771, 6599, 5566, 8129, 6950],
 [4985, 13220, 1594, 11154, 4188, 1865, 3257, 15165, 16528

In [24]:
max_sentence_len = 30
padded_one_hot_rep = pad_sequences(one_hot_rep, maxlen = max_sentence_len, padding = 'pre')
padded_one_hot_rep[:50]

array([[    0,     0,     0, ...,  1540,  4706,  4532],
       [    0,     0,     0, ..., 12227, 19888, 14885],
       [    0,     0,     0, ..., 16444, 11841, 17729],
       ...,
       [    0,     0,     0, ...,  2943, 10959, 12137],
       [    0,     0,     0, ...,  1608,  8171,  2689],
       [    0,     0,     0, ...,  5566,  8129,  6950]], dtype=int32)

In [25]:
padded_one_hot_rep.shape

(18285, 30)

In [27]:
from keras import layers, models

embedding_features = 50

input_layer = layers.Input(shape = (max_sentence_len,), name = 'input_layer')
x = layers.Embedding(vocab_size, output_dim = embedding_features, name = 'embedding_layer')(input_layer)
x = layers.Dropout(0.3)(x)
x = layers.Bidirectional(layers.LSTM(128, name = 'bidirectional_lstm'))(x)
x = layers.Dropout(0.3)(x)
output_layer = layers.Dense(1, activation = 'sigmoid', name = 'output_layer')(x)

model = models.Model(inputs = input_layer, outputs = output_layer, name = 'single-lstm-dropout')


In [28]:
model.summary()

Model: "single-lstm-dropout"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 30)]              0         
                                                                 
 embedding_layer (Embedding  (None, 30, 50)            1000000   
 )                                                               
                                                                 
 dropout_2 (Dropout)         (None, 30, 50)            0         
                                                                 
 bidirectional (Bidirection  (None, 256)               183296    
 al)                                                             
                                                                 
 dropout_3 (Dropout)         (None, 256)               0         
                                                                 
 output_layer (Dense)        (None, 1)         

In [29]:
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss = tf.keras.losses.BinaryCrossentropy(),
              metrics = ['accuracy'])

In [30]:
pad_embedd_X_train = np.array(padded_one_hot_rep)
y_train_arr = np.array(y_train)

In [31]:
pad_embedd_X_train.shape

(18285, 30)

In [32]:
y_train_arr.shape

(18285,)

In [33]:
epochs = 10
batch_size = 256

from sklearn.model_selection import train_test_split

(X_train, X_test, y_train, y_test) = train_test_split(pad_embedd_X_train, y_train_arr, test_size = 0.1, random_state = 42)

model.fit(X_train, y_train, epochs = epochs, batch_size = batch_size, validation_split = 0.1, verbose = 1)

Epoch 1/10
58/58 [==============================] - 16s 155ms/step - loss: 0.5228 - accuracy: 0.7155 - val_loss: 0.2465 - val_accuracy: 0.8943
Epoch 2/10
58/58 [==============================] - 6s 98ms/step - loss: 0.1669 - accuracy: 0.9361 - val_loss: 0.1788 - val_accuracy: 0.9228
Epoch 3/10
58/58 [==============================] - 3s 46ms/step - loss: 0.0995 - accuracy: 0.9653 - val_loss: 0.1898 - val_accuracy: 0.9253
Epoch 4/10
58/58 [==============================] - 3s 56ms/step - loss: 0.0698 - accuracy: 0.9765 - val_loss: 0.2084 - val_accuracy: 0.9235
Epoch 5/10
58/58 [==============================] - 2s 39ms/step - loss: 0.0500 - accuracy: 0.9833 - val_loss: 0.2364 - val_accuracy: 0.9204
Epoch 6/10
58/58 [==============================] - 3s 52ms/step - loss: 0.0376 - accuracy: 0.9881 - val_loss: 0.2627 - val_accuracy: 0.9235
Epoch 7/10
58/58 [==============================] - 2s 29ms/step - loss: 0.0266 - accuracy: 0.9920 - val_loss: 0.3251 - val_accuracy: 0.9216
Epoch 8/10


In [34]:
y_pred = model.predict(X_test)

58/58 [==============================] - 1s 4ms/step


In [35]:
model.evaluate(X_test, y_test)

58/58 [==============================] - 0s 4ms/step - loss: 0.5453 - accuracy: 0.9005


[0.5452996492385864, 0.9004920721054077]

In [45]:
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred.round())

print(f"Mean Squared Error = {mse}")
print(f"R2 Score = {r2}")
print(f"Accuracy = {round(accuracy*100, 4)} %")


Mean Squared Error = 0.08807920698067043
R2 Score = 0.6413524536129052
Accuracy = 90.0492 %


In [41]:
from sklearn.metrics import confusion_matrix, classification_report

print(classification_report(y_test,y_pred.round()))

              precision    recall  f1-score   support

           0       0.91      0.91      0.91      1036
           1       0.89      0.89      0.89       793

    accuracy                           0.90      1829
   macro avg       0.90      0.90      0.90      1829
weighted avg       0.90      0.90      0.90      1829



In [46]:
model.save("bidirectional-lstm-fake-news-detector.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
